In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
PATH = 'C:\\Users\\arsener\\PycharmProjects\\tensorflow2.0beta\\zhihu\\data\\'

(train_x, train_y), (test_x, test_y) = tf.keras.datasets.imdb.load_data(path=PATH + 'imdb.npz', num_words=10000)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((25000,), (25000,), (25000,), (25000,))

In [3]:
word_index = tf.keras.datasets.imdb.get_word_index(path=PATH + 'imdb_word_index.json')

In [4]:
word2id = {k:(v+3) for k, v in word_index.items()}
word2id['<PAD>'] = 0
word2id['<START>'] = 1
word2id['<UNK>'] = 2
word2id['<UNUSED>'] = 3

id2word = {v:k for k, v in word2id.items()}
def get_words(sent_ids):
    return ' '.join([id2word.get(i, '?') for i in sent_ids])

sent = get_words(train_x[0])
print(sent)

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [5]:
# 句子末尾padding
LENGTH = 256

train_x = tf.keras.preprocessing.sequence.pad_sequences(
    train_x, value=word2id['<PAD>'],
    padding='post', maxlen=LENGTH
)
test_x = tf.keras.preprocessing.sequence.pad_sequences(
    test_x, value=word2id['<PAD>'],
    padding='post', maxlen=LENGTH
)
print(train_x[0])
print('len: ',len(train_x[0]), len(train_x[1]))

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [6]:
TRAIN_NUM = train_x.shape[0]

gap = int(0.8 * TRAIN_NUM)

x_train = train_x[:gap]
x_val = train_x[gap:]

y_train = train_y[:gap]
y_val = train_y[gap:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

BATCH_SIZE = 128
train_dataset = train_dataset.repeat().shuffle(gap).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [7]:
vocab_size = 10000
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=LENGTH),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 256, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                65552     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 225,569
Trainable params: 225,569
Non-trainable params: 0
_________________________________________________________________


In [8]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        # 是否有提升关注的指标
        monitor='val_accuracy',
        # 不再提升的阈值
        min_delta=1e-2,
        # 2个epoch没有提升就停止
        patience=3,
        verbose=1
    )
]

history = model.fit(train_dataset,
                   epochs=40,
                   steps_per_epoch=int(np.ceil(gap / float(BATCH_SIZE))),
                   validation_data=val_dataset,
                   validation_steps=int(np.ceil((TRAIN_NUM - gap) / float(BATCH_SIZE))),
                   callbacks=callbacks,
                   verbose=1)

Epoch 1/40


W0728 22:05:48.001500 15748 deprecation.py:323] From C:\Users\arsener\PycharmProjects\tensorflow2.0beta\venv\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


157/157 [==============================] - 4s 23ms/step - loss: 0.5787 - accuracy: 0.6728 - val_loss: 0.3851 - val_accuracy: 0.8194
Epoch 2/40
157/157 [==============================] - 3s 19ms/step - loss: 0.2345 - accuracy: 0.9111 - val_loss: 0.3007 - val_accuracy: 0.8722
Epoch 3/40
157/157 [==============================] - 3s 18ms/step - loss: 0.1294 - accuracy: 0.9590 - val_loss: 0.3154 - val_accuracy: 0.8708
Epoch 4/40
157/157 [==============================] - 3s 19ms/step - loss: 0.0614 - accuracy: 0.9846 - val_loss: 0.3523 - val_accuracy: 0.8664
Epoch 5/40
157/157 [==============================] - 3s 19ms/step - loss: 0.0267 - accuracy: 0.9963 - val_loss: 0.3656 - val_accuracy: 0.8708
Epoch 00005: early stopping


In [10]:
[key for key in history.history]

['loss', 'accuracy', 'val_loss', 'val_accuracy']

In [14]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 3))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [15]:
result = model.evaluate(test_x, test_y)

25000/25000 [==============================] - 1s 44us/sample - loss: 0.3908 - accuracy: 0.8632


In [ ]:
'''
为什么和原样例比，在验证集和测试集上的acc均小1%？
'''